In [1]:
import tensorflow as tf

#reset tensorflow graph
tf.reset_default_graph()

#Import Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#Training & Test Data
trainX = mnist.train.images
trainY = mnist.train.labels



Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
import numpy as np
import pandas as pd
import timeit
import tensorflow as tf
from sklearn.cross_validation import train_test_split

D:\Work\Anaconda3\envs\tensorflow-cpu\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
start = timeit.default_timer()

In [4]:
trainX,testX,trainY,testY=train_test_split(trainX, trainY, test_size=0.20, random_state=42)

In [5]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(44000, 784)
(44000, 10)
(11000, 784)
(11000, 10)


In [6]:
#trainX=trainX/255
#testX=testX/255

In [7]:
#Classifier Parameters
logs_path='/tmp/session45/v13'
save_path=logs_path + '/'
learning_rate = 0.003 #0.003, 0.0001, 0.0002
n_features = 784
n_classes = 10
batch_size = 100
training_epochs = 160

In [8]:
## Defining Hidden Layers
K = 400
L = 300
M = 200
N = 100
O=50

In [9]:
with tf.name_scope('input'):
    # None -> batch size can be any size, with n_features
    x = tf.placeholder(tf.float32, shape=[None, n_features], name="x-input") 
    # target n_classes output classes
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")

In [10]:
## Defining Swish Activation
def swish(x):
    return x*tf.nn.sigmoid(x)

In [11]:
def swish(x):
    beta=tf.Variable(initial_value=1.0,trainable=True,name='swish')
    return x*tf.nn.sigmoid(beta*x) #trainable parameter beta


In [12]:
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.truncated_normal([n_features, K] ,stddev=0.1))
    b1 = tf.Variable(tf.ones([K]))
    #Y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)
    #Y1 = tf.nn.relu(tf.matmul(x, W1) + b1)
    Y1 = swish(tf.add(tf.matmul(x, W1),b1))

In [13]:
with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.truncated_normal([K, L] ,stddev=0.1))
    b2 = tf.Variable(tf.ones([L]))
    #Y2 = tf.nn.sigmoid(tf.matmul(Y1, W2) + b2)
    #Y2 = tf.nn.relu(tf.matmul(Y1, W2) + b2)
    Y2 = swish(tf.add(tf.matmul(Y1, W2), b2))

In [14]:
with tf.name_scope("Layer3"):
    W3 = tf.Variable(tf.truncated_normal([L, M] ,stddev=0.1))
    b3 = tf.Variable(tf.ones([M]))
    #Y3 = tf.nn.sigmoid(tf.matmul(Y2, W3) + b3)
    #Y3 = tf.nn.relu(tf.matmul(Y2, W3) + b3)
    Y3 = swish(tf.add(tf.matmul(Y2, W3), b3))

In [15]:
with tf.name_scope("Layer4"):
    W4 = tf.Variable(tf.truncated_normal([M, N] ,stddev=0.1))
    b4 = tf.Variable(tf.ones([N]))
    #Y4 = tf.nn.sigmoid(tf.matmul(Y3, W4) + b4)
    #Y4 = tf.nn.relu(tf.matmul(Y3, W4) + b4)
    Y4 = swish(tf.add(tf.matmul(Y3, W4), b4))

In [16]:
with tf.name_scope("Layer5"):
    W5=tf.Variable(tf.truncated_normal([N,O], stddev=0.1))
    b5=tf.Variable(tf.ones([O]))
    Y5=swish(tf.add(tf.matmul(Y4,W5),b5))

In [17]:
# implement model
with tf.name_scope("Output"):
    # y is our prediction
    W = tf.Variable(tf.truncated_normal([O, n_classes] ,stddev=0.1))
    b = tf.Variable(tf.zeros([n_classes]))   
    #y = tf.nn.softmax(tf.matmul(Y5, W) + b)
    Ylogits = tf.matmul(Y5, W) + b
    y = tf.nn.softmax(Ylogits)

In [18]:
# specify cost function
with tf.name_scope('Loss'):
    # this is our cost
    #cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=y_)
    cross_entropy = tf.reduce_mean(cross_entropy)*100

In [19]:
# specify optimizer
with tf.name_scope('train'):
    # optimizer is an "operation" which we can execute in a session
    learn_rate = tf.Variable(learning_rate)
    train_op = tf.train.AdadeltaOptimizer(learn_rate).minimize(cross_entropy)

In [20]:
with tf.name_scope('Accuracy'):
    # Prediction
    prediction = tf.argmax(y,1,name="Predict")
    #Accuracy
    correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")

In [21]:
# create a summary for our cost and accuracy
training_loss = tf.summary.scalar("training_loss", cross_entropy)
training_accuracy = tf.summary.scalar("training_accuracy", accuracy)
test_loss = tf.summary.scalar("test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("test_accuracy", accuracy)

In [22]:
#Create a Saver to save the graph
saver = tf.train.Saver()

In [23]:
#Start Graph execution
with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # perform training cycles
    for epoch in range(training_epochs):
        
        # number of batches in one epoch
        batch_count = int(trainX.shape[0]/batch_size)
        
        for i in range(batch_count):
            batch_x  = trainX[i*batch_size:i*batch_size+batch_size]
            batch_y  = trainY[i*batch_size:i*batch_size+batch_size]

            # perform the operations we defined earlier on batch
            _,acc,loss = sess.run([train_op, training_accuracy,training_loss], feed_dict={x: batch_x, y_: batch_y})
            
            #log training accuracy and loss
            writer.add_summary(acc, epoch * batch_count + i)
            writer.add_summary(loss, epoch * batch_count + i)    
                        
        #Test loss and accuracy
        acc,loss,a_loss = sess.run([test_accuracy,test_loss,cross_entropy],
                                   feed_dict={x: testX, y_: testY})
        writer.add_summary(acc, epoch * batch_count + i)
        writer.add_summary(loss, epoch * batch_count + i)
        print ('Test loss at step: ', epoch, ' is ', a_loss) 
        if epoch % 5 == 0: 
            print ("Epoch: ", epoch)
                
    print ("Accuracy: ", accuracy.eval(feed_dict={x: testX, y_: testY}))
    
    #Save the model
    saver.save(sess, save_path + "model.ckpt")
    print ("done")

Test loss at step:  0  is  250.778
Epoch:  0
Test loss at step:  1  is  214.695
Test loss at step:  2  is  193.717
Test loss at step:  3  is  176.513
Test loss at step:  4  is  161.131
Test loss at step:  5  is  147.319
Epoch:  5
Test loss at step:  6  is  134.91
Test loss at step:  7  is  123.767
Test loss at step:  8  is  113.819
Test loss at step:  9  is  105.02
Test loss at step:  10  is  97.3112
Epoch:  10
Test loss at step:  11  is  90.5978
Test loss at step:  12  is  84.7638
Test loss at step:  13  is  79.6909
Test loss at step:  14  is  75.2691
Test loss at step:  15  is  71.401
Epoch:  15
Test loss at step:  16  is  68.0024
Test loss at step:  17  is  65.0017
Test loss at step:  18  is  62.3385
Test loss at step:  19  is  59.9627
Test loss at step:  20  is  57.8322
Epoch:  20
Test loss at step:  21  is  55.9122
Test loss at step:  22  is  54.1737
Test loss at step:  23  is  52.5927
Test loss at step:  24  is  51.149
Test loss at step:  25  is  49.8258
Epoch:  25
Test loss at s

In [24]:
stop = timeit.default_timer()

print (stop - start)

665.3527445754144
